In [ ]:
import pandas as pd
data = pd.read_csv('./Data/20240205fullsample_new.csv')
data.head()

The purpose of `installment_timestep` is to generate a timestep for each installment of each loan (loan id), indicating the order in which that installment is repaid in the current loan.

In [22]:
data['installment_timestep'] = data.groupby(
    ['loan_id', 'installment']).cumcount() + 1

In [ ]:
data_sim = data.loc[data['sample'] == 'rlsimulator']
data_sim

In [40]:
# specific columns names
loan_id = ['loan_id']
user_features = ['gender',
                 'age',
                 'amount',
                 'num_loan',
                 'duration',
                 'year_ratio',
                 'diff_city',
                 'marriage',
                 'kids',
                 'month_in',
                 'housing',
                 'edu',
                 'motivation']
current_state = ['installment',
                 'installment_timestep',
                 'state_cum_overduelength',
                 'remaining_debt',
                 'state_capital',
                 'state_interests',
                 'state_penalty',
                 ]
other_labels = ['installment_done',
                'loan_done',
                'recovery_rate_weighted']

In [ ]:
loan_id_list = data_sim['loan_id'].unique().tolist()
# len(loan_id_list)
# loan_id_list
target_state = pd.DataFrame()


col_matching = {'installment': 'y_installment',
                'installment_timestep': 'y_installment_timestep',
                'state_cum_overduelength': 'y_state_cum_overduelength',
                'remaining_debt': 'y_remaining_debt',
                'state_capital': 'y_state_capital',
                'state_interests': 'y_state_interests',
                'state_penalty': 'y_state_penalty'}


for example_id in loan_id_list:
    example_data = data_sim.loc[data_sim['loan_id'] == example_id]
    y_train = pd.DataFrame()
    y_train = example_data[current_state]
    y_train = y_train.rename(columns=col_matching)

    if y_train.shape[0] > 1:
        y_train = y_train[1:]
        y_train = pd.concat(
            [y_train, y_train.iloc[[-1]]], ignore_index=True)
    target_state = pd.concat([target_state, y_train], ignore_index=True)


target_state

In [ ]:
data_sim_full = pd.DataFrame()
data_sim_full = pd.concat(
    [data_sim[['group']], data_sim[loan_id+user_features+current_state], target_state, data_sim[other_labels]], axis=1)
data_sim_full

In [57]:
data_sim_full.to_csv('./Res/simulator_data.csv', index=False)

In [59]:
data_sim_full.to_excel('./Res/simulator_data.xlsx', index=False)